In [1]:
!pip install finterstellar

In [2]:
# Import library
import finterstellar as fs
#System trading Library 설치: Finterstellar

import io, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
symbol_ = 'TSLA'
start_date_ = '2020-06-30'
end_date_ = '2025-06-30'
fee_ = 0.001

In [4]:
import os

path_list = ["rsi","macd","stochastic"]

for path in path_list :
    try:
        os.makedirs(path, exist_ok=True)
        print(f"{path} 디렉토리가 생성되었거나 이미 존재합니다.")
    except OSError as e:
        print(f"디렉토리 생성 중 오류 발생: {e}")


rsi 디렉토리가 생성되었거나 이미 존재합니다.
macd 디렉토리가 생성되었거나 이미 존재합니다.
stochastic 디렉토리가 생성되었거나 이미 존재합니다.


In [5]:
# 1. RSI Performance and evaluation

In [6]:
# finstaller.performance 를 dict 로 반환하는 함수

import re

def parse_performance_output(output_text):
    result = {}
    for line in output_text.strip().split("\n"):
        # 콜론으로 분리
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = key.strip()
        value = value.strip()

        # 퍼센트 처리
        if value.endswith("%"):
            num = float(value.strip("%")) / 100
            result[key] = num
        # 연도 처리
        elif value.endswith("yrs"):
            num = float(value.strip("yrs"))
            result[key] = num
        # 숫자 처리
        else:
            try:
                result[key] = float(value)
            except ValueError:
                result[key] = value  # 혹시 문자열이면 그대로 저장

    return result


def parse_performance_output(output_text, pre_text):
    result = {}
    for line in output_text.strip().split("\n"):
        # 콜론으로 분리
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = pre_text + "_" + key.strip()
        value = value.strip()

        # 퍼센트 처리
        if value.endswith("%"):
            num = float(value.strip("%")) / 100
            result[key] = num
        # 연도 처리
        elif value.endswith("yrs"):
            num = float(value.strip("yrs"))
            result[key] = num
        # 숫자 처리
        else:
            try:
                result[key] = float(value)
            except ValueError:
                result[key] = value  # 혹시 문자열이면 그대로 저장

    return result

In [7]:
# RSI 함수 정의
def get_rsi_performance(df, w_, buy_, sell_, fee) :
    fs.rsi(df, w=w_)
    fs.indicator_to_signal(df, factor='rsi', buy=buy_, sell=sell_)
    fs.position(df)

    # Position 유효성 검사
    if "position" not in df :
        print(f"* None of position column as w : {w_}, buy : {buy_}, sell : {sell_}")


    fs.evaluate(df, cost= fee)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # 함수 실행 (print만 함)
        fs.performance(df, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    performance_dict = parse_performance_output(output_text, 'RSI')

    return performance_dict

In [8]:
# MACD 함수 정의
def get_macd_performance(df, buy_, sell_, fee) :
    fs.macd(df)
    fs.indicator_to_signal(df, factor='macd', buy=buy_, sell=sell_)
    fs.position(df)
    fs.evaluate(df, cost= fee)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # 함수 실행 (print만 함)
        fs.performance(df, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    performance_dict = parse_performance_output(output_text, 'MACD')

    return performance_dict

In [9]:
# Stochastics 함수 정의
# Stochastics 함수는 w_.값에 factor ('slow_k', 'indicator') 값을 이용한다
def get_stochastics_performance(symbol, df, factor_, buy_, sell_, fee_) :
    fs.stochastic(df, symbol, n=14, m=3, t=3)
    # indicator 값 추출을 위한 전처리 작업
    df['indicator'] = df['slow_k'] - df['slow_d']

    fs.indicator_to_signal(df, factor= factor_, buy=buy_, sell=sell_)
    fs.position(df)
    fs.evaluate(df, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # 함수 실행 (print만 함)
        fs.performance(df, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    performance_dict = parse_performance_output(output_text, 'Stochastics')

    return performance_dict

In [28]:
# 기본 팩터자료 함수 정의
# symbol : 주식 약칭명
# start_date_ : 시작일자, end_date_ : 종료일자
# fee_ : 거래수수료
def get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_) :
    import importlib
    
    
    # df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    results = []
    # 최적의 w, buy_position, sell_position 을 찾기 위한 recursive
    for w_ in range(7, 23+1) :
        for buy_ in range(15, 45+1) :
            for sell_ in range(55, 85+1) :
                try :
                    # library re-import
                    import finterstellar as fs
                    importlib.reload(fs)
                    
                    # dataFrame 초기화
                    df_tmp = df.copy()
                    
                    result_dict = {}

                    # 파라미터 추가
                    result_dict["symbol"] = symbol
                    result_dict["w_param"] = w_
                    result_dict["buy_param"] = buy_
                    result_dict["sell_param"] = sell_
                    result_dict["fee_"] = fee_

                    perf = {}

                    perf = get_rsi_performance(df_tmp, w_, buy_, sell_, fee_)

                    result_dict.update(perf)

                    # 리스트에 추가
                    results.append(result_dict)

                    if(len(results) % 50 == 0) :
                        print(f"Performance Processing : [{len(results)}] {result_dict}")
                except Exception as e :
                    print(" !!! Error : ",e)
                    continue

    df_result = pd.DataFrame(results)
    return df_result


In [11]:
# 기본 팩터자료 함수 정의
# symbol : 주식 약칭명
# start_date_ : 시작일자, end_date_ : 종료일자
# fee_ : 거래수수료
def get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_) :
    # df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)
    
    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    results = []
    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    for buy_ in range(-10, 10+1) :
        for sell_ in range(-10, 10+1) :
            try :
                result_dict = {}

                # 파라미터 추가
                result_dict["symbol"] = symbol
                result_dict["buy_param"] = buy_
                result_dict["sell_param"] = sell_
                result_dict["fee_"] = fee_

                perf = {}

                perf = get_macd_performance(df, buy_, sell_, fee_)

                result_dict.update(perf)

                # 리스트에 추가
                results.append(result_dict)

                if(len(results) % 100 == 0) :
                    print(f"Performance Processing : [{len(results)}] {result_dict}")
            except Exception as e :
                print(" !!! Error : ",e)
                continue

    df_result = pd.DataFrame(results)
    return df_result


In [12]:
# 기본 팩터자료 함수 정의
# symbol : 주식 약칭명
# start_date_ : 시작일자, end_date_ : 종료일자
# fee_ : 거래수수료
def get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_) :
    # df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)
    df = pd.read_csv(f"get_ohlc_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)
    
    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    results = []
    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    # 1. slow_k
    for buy_ in range(10, 40+1) :
        for sell_ in range(60, 90+1) :
            try :
                result_dict = {}

                # 파라미터 추가
                result_dict["symbol"] = symbol
                result_dict["factor"] = "slow_k"
                result_dict["buy_param"] = buy_
                result_dict["sell_param"] = sell_
                result_dict["fee_"] = fee_

                perf = {}

                perf = get_stochastics_performance(symbol, df, "slow_k", buy_, sell_, fee_)

                result_dict.update(perf)

                # 리스트에 추가
                results.append(result_dict)
                if(len(results) % 500 == 0) :
                    print(f"Performance Processing : [{len(results)}] {result_dict}")
            except Exception as e :
                print(" !!! Error : ",e)
                continue


    # 2. indicator
    for buy_ in range(-10, 10+1) :
        for sell_ in range(-10, 10+1) :
            try :
                result_dict = {}

                # 파라미터 추가
                result_dict["symbol"] = symbol
                result_dict["factor"] = "indicator"
                result_dict["buy_param"] = buy_
                result_dict["sell_param"] = sell_
                result_dict["fee_"] = fee_

                perf = {}

                perf = get_stochastics_performance(symbol, df, "indicator", buy_, sell_, fee_)

                result_dict.update(perf)

                # 리스트에 추가
                results.append(result_dict)

                if(len(results) % 500 == 0) :
                    print(f"Performance Processing : [{len(results)}] {result_dict}")
            except Exception as e :
                print(" !!! Error : ",e)
                continue

    df_result = pd.DataFrame(results)

    return df_result


In [13]:
def get_rsi_algorithm_info(symbol, start_date_, end_date_, w_, buy_, sell_, text, fee_) :
    # df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # rsi
    fs.rsi(df, w=w_)
    fs.draw_chart(df, left='rsi', right= symbol)
    # 파일로 저장
    plt.savefig(f"./rsi//{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='rsi', buy=buy_, sell=sell_)
    # Calc position
    fs.position(df)
    fs.draw_chart(df, left='rsi', right='position_chart')
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # Evalution
    fs.evaluate(df, cost= fee_)
    fs.draw_chart(df, left='acc_rtn_dp', right=symbol)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_evaluation_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # get Performance
    fs.performance(df, rf_rate=0.02)

    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [14]:
def get_macd_algorithm_info(symbol, start_date_, end_date_, buy_, sell_,text,  fee_) :
    # df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # macd
    fs.macd(df)
    fs.draw_chart(df, right=['macd','macd_signal','macd_oscillator'])
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='macd_oscillator', buy=buy_, sell=sell_)

    # Calc position
    fs.position(df)
    fs.draw_chart(df, right='position_chart', left='macd_oscillator')
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [15]:
def get_stochastic_algorithm_info(symbol, start_date_, end_date_, factor_, buy_, sell_,text, fee_) :
    # df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)
    df = pd.read_csv(f"get_ohlc_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # stochastic
    fs.stochastic(df, symbol, n=14, m=3, t=3)
    df['indicator'] = df['slow_k'] - df['slow_d']
    fs.draw_chart(df, left=factor_, right=symbol)
    # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor=factor_, buy=buy_, sell=sell_)

    # Calc position
    fs.position(df)
    fs.draw_chart(df, right='position_chart', left='slow_k')
    # # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # # 리소스 해제
    # plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [16]:
# # Max CAGR
# df_max_cagr = df_rsi[df_rsi['RSI_CAGR'] == df_rsi['RSI_CAGR'].max()]
# df_max_cagr.iloc[0]

In [17]:
# # Max Sharpe ratio
# df_max_sharpe_ratio = df_rsi[df_rsi['RSI_Sharpe ratio'] == df_rsi['RSI_Sharpe ratio'].max()]
# df_max_sharpe_ratio.iloc[0]

In [18]:
# get_rsi_algorithm_info(df_max_cagr.iloc[0]['symbol'],start_date_,end_date_,df_max_cagr.iloc[0]['w_param'],df_max_cagr.iloc[0]['buy_param'],df_max_cagr.iloc[0]['sell_param'], fee_)

In [19]:
def process(symbol, start_date_, end_date_, fee_) :
    df_rsi = get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_macd = get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_sto = get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_)

    # write csv
    df_rsi.to_csv(f'./rsi/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)
    df_macd.to_csv(f'./macd/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)
    df_sto.to_csv(f'./stochastic/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

    # find max value
    # RSI CAGR
    df_rsi_max_cagr = df_rsi[df_rsi['RSI_CAGR'] == df_rsi['RSI_CAGR'].max()].iloc[0]
    get_rsi_algorithm_info(df_rsi_max_cagr['symbol'],start_date_,end_date_,df_rsi_max_cagr['w_param'],df_rsi_max_cagr['buy_param'],df_rsi_max_cagr['sell_param'],'CAGR', fee_)

    # RSI Sharpe ratio
    df_rsi_max_sharpe_ratio = df_rsi[df_rsi['RSI_Sharpe ratio'] == df_rsi['RSI_Sharpe ratio'].max()].iloc[0]
    get_rsi_algorithm_info(df_rsi_max_sharpe_ratio['symbol'],start_date_,end_date_,df_rsi_max_sharpe_ratio['w_param'],df_rsi_max_sharpe_ratio['buy_param'],df_rsi_max_sharpe_ratio['sell_param'],'SharpeRatio', fee_)

    # MACD CAGR
    df_macd_max_cagr = df_macd[df_macd['MACD_CAGR'] == df_macd['MACD_CAGR'].max()].iloc[0]
    get_macd_algorithm_info(df_macd_max_cagr['symbol'],start_date_,end_date_,df_macd_max_cagr['buy_param'],df_macd_max_cagr['sell_param'],'CAGR', fee_)

    # MACD Sharpe ratio
    df_macd_max_sharpe_ratio = df_macd[df_macd['MACD_Sharpe ratio'] == df_macd['MACD_Sharpe ratio'].max()].iloc[0]
    get_macd_algorithm_info(df_macd_max_sharpe_ratio['symbol'],start_date_,end_date_,df_macd_max_sharpe_ratio['buy_param'],df_macd_max_sharpe_ratio['sell_param'],'SharpeRatio', fee_)

    # Stochastic CAGR
    df_sto_max_cagr = df_sto[df_sto['Stochastics_CAGR'] == df_sto['Stochastics_CAGR'].max()].iloc[0]
    get_stochastic_algorithm_info(df_sto_max_cagr['symbol'],start_date_,end_date_,df_sto_max_cagr['factor'],df_sto_max_cagr['buy_param'],df_sto_max_cagr['sell_param'],'CAGR', fee_)

    # Stochastic Sharpe ratio
    df_sto_max_sharpe_ratio = df_sto[df_sto['Stochastics_Sharpe ratio'] == df_sto['Stochastics_Sharpe ratio'].max()].iloc[0]
    get_stochastic_algorithm_info(df_sto_max_sharpe_ratio['symbol'],start_date_,end_date_,df_sto_max_sharpe_ratio['factor'],df_sto_max_sharpe_ratio['buy_param'],df_sto_max_sharpe_ratio['sell_param'],'SharpeRatio', fee_)

    # return max value
    return df_rsi_max_cagr, df_rsi_max_sharpe_ratio, df_macd_max_cagr, df_macd_max_sharpe_ratio, df_sto_max_cagr, df_sto_max_sharpe_ratio

In [20]:
symbol_list = ['^GSPC', '^KS11','MSFT', 'GOOG', 'AMZN', 'META', 'NVDA', 'TSLA', 'AAPL']

In [ ]:
# for
for symbol in symbol_list :
    df_rsi_max_cagr, df_rsi_max_sharpe_ratio, df_macd_max_cagr, df_macd_max_sharpe_ratio, df_sto_max_cagr, df_sto_max_sharpe_ratio \
        = process(symbol, start_date_, end_date_, fee_)

    print(f"##### [{symbol}] 1. RSI #####")
    print(f"   ***** (1) CAGR *****")
    print(df_rsi_max_cagr)
    print(f"   ***** (2) Sharpe Ratio *****")
    print(df_rsi_max_sharpe_ratio)

    print(f"##### [{symbol}] 2. MACD #####")
    print(f"   ***** (1) CAGR *****")
    print(df_macd_max_cagr)
    print(f"   ***** (2) Sharpe Ratio *****")
    print(df_macd_max_sharpe_ratio)

    print(f"##### [{symbol}] 3. Stochastic #####")
    print(f"   ***** (1) CAGR *****")
    print(df_sto_max_cagr)
    print(f"   ***** (2) Sharpe Ratio *****")
    print(df_sto_max_sharpe_ratio)
    print("\n\n")

In [39]:
w_range = range(7,23)
buy_range = range(15,45)
sell_range = range(55,85)
param_list = [(w,buy,sell,fee_) for w in w_range for buy in buy_range for sell in sell_range]
print(param_list)

[(7, 15, 55, 0.001), (7, 15, 56, 0.001), (7, 15, 57, 0.001), (7, 15, 58, 0.001), (7, 15, 59, 0.001), (7, 15, 60, 0.001), (7, 15, 61, 0.001), (7, 15, 62, 0.001), (7, 15, 63, 0.001), (7, 15, 64, 0.001), (7, 15, 65, 0.001), (7, 15, 66, 0.001), (7, 15, 67, 0.001), (7, 15, 68, 0.001), (7, 15, 69, 0.001), (7, 15, 70, 0.001), (7, 15, 71, 0.001), (7, 15, 72, 0.001), (7, 15, 73, 0.001), (7, 15, 74, 0.001), (7, 15, 75, 0.001), (7, 15, 76, 0.001), (7, 15, 77, 0.001), (7, 15, 78, 0.001), (7, 15, 79, 0.001), (7, 15, 80, 0.001), (7, 15, 81, 0.001), (7, 15, 82, 0.001), (7, 15, 83, 0.001), (7, 15, 84, 0.001), (7, 16, 55, 0.001), (7, 16, 56, 0.001), (7, 16, 57, 0.001), (7, 16, 58, 0.001), (7, 16, 59, 0.001), (7, 16, 60, 0.001), (7, 16, 61, 0.001), (7, 16, 62, 0.001), (7, 16, 63, 0.001), (7, 16, 64, 0.001), (7, 16, 65, 0.001), (7, 16, 66, 0.001), (7, 16, 67, 0.001), (7, 16, 68, 0.001), (7, 16, 69, 0.001), (7, 16, 70, 0.001), (7, 16, 71, 0.001), (7, 16, 72, 0.001), (7, 16, 73, 0.001), (7, 16, 74, 0.001),

In [53]:
from joblib import Parallel, delayed
from tqdm import tqdm

# 기본 팩터자료 함수 정의
# symbol : 주식 약칭명
# start_date_ : 시작일자, end_date_ : 종료일자
# fee_ : 거래수수료
def get_rsi_performance_dataframe_parallel(symbol, start_date_, end_date_, fee_) :
    
    # df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))
    
    w_range = range(7,23)
    buy_range = range(15,45)
    sell_range = range(55,85)
    param_list = [(w,buy,sell,fee_) for w in w_range for buy in buy_range for sell in sell_range]
    print(param_list[0])
    
    
    results = []
    
    # 최적의 w, buy_position, sell_position 을 찾기 위한 recursive
    # Parallel 이용
    results = Parallel(n_jobs=-1) (
            delayed(get_rsi_performance_process)(df, params) for params in tqdm(param_list)
            
    )
    
    df_result = pd.DataFrame(results)
    return df_result

def get_rsi_performance_process(df, params) :
    w_, buy_, sell_, fee_ = params
    
    # dataFrame 초기화
    df_tmp = df.copy()
    
    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol
    result_dict["w_param"] = w_
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_
    result_dict["fee_"] = fee_

    perf = {}

    perf = get_rsi_performance(df_tmp, w_, buy_, sell_, fee_)

    result_dict.update(perf)

    print(f"Performance Processing : {result_dict}")

    return result_dict


In [55]:
result = get_rsi_performance_dataframe_parallel('TSLA', start_date_, end_date_, fee_)

[ TSLA ] dataFrame.head 
            TSLA
2020-06-30 71.99
2020-07-01 74.64
2020-07-02 80.58
(7, 15, 55, 0.001)


100%|██████████| 14400/14400 [1:22:37<00:00,  2.90it/s]


In [56]:
result.head()

,symbol,w_param,buy_param,sell_param,fee_,RSI_CAGR,RSI_Accumulated return,RSI_Average return,RSI_Benchmark return,RSI_Number of trades,RSI_Number of win,RSI_Hit ratio,RSI_Investment period,RSI_Sharpe ratio,RSI_MDD,RSI_Benchmark MDD
0,^GSPC,7,15,55,0.00,0.04,0.23,0.03,2.42,7.00,4.00,0.57,5.00,0.08,-0.25,-0.74
1,^GSPC,7,15,56,0.00,0.04,0.23,0.03,2.42,7.00,4.00,0.57,5.00,0.08,-0.25,-0.74
2,^GSPC,7,15,57,0.00,0.04,0.21,0.03,2.42,7.00,3.00,0.43,5.00,0.06,-0.25,-0.74
3,^GSPC,7,15,58,0.00,0.04,0.21,0.03,2.42,7.00,3.00,0.43,5.00,0.06,-0.25,-0.74
4,^GSPC,7,15,59,0.00,0.05,0.28,0.04,2.42,7.00,3.00,0.43,5.00,0.10,-0.25,-0.74


In [59]:
result.to_csv(f'./rsi/TSLA_RSI.csv',encoding='utf8', index=False)